# Imports

In [31]:
from bonsai.nas import Bonsai
from bonsai.helpers import *
from bonsai.data_loaders import *
from bonsai.net import Net
from bonsai.ops import Classifier
import torch
import torch.nn as nn
%load_ext autoreload
%autoreload 2

mem_stats()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'   3.21GiB'

# Define Hypers

In [3]:
hypers = {
    'gpu_space':10.5,
    'dataset':{'name':'CIFAR10', 'classes':4},
    'batch_size':64,
    'scale':36,
    'nodes':4,
    'depth':4,
    'patterns':[['n','na'],['r']],
    'post_patterns':1,
    'reduction_target':2,
    'lr_schedule': {'lr_max': .01, 'T': 600},
    'drop_prob':.3,
    'nas_schedule': {'prune_interval':4, 'cycle_len':8},
    'prune_rate':{'edge':.1, 'input':.1}
}

# Run

In [8]:
bonsai = Bonsai(hypers, sizes={
    1:0.8295454545454546,
    2:0.40340909090909083,
    3:0.5170454545454545,
    4:0.40340909090909083,
    },
start_size=1)
bonsai.random = 3
bonsai.e_c = [.387]
bonsai.i_c = [1.0]
bonsai.generate_model()

Generating model at random level 3, e_c=0.387, i_c=1.0, prune=False
torch.Size([64, 3, 32, 32])


In [9]:
import matplotlib.pyplot as plt

class_dls = bonsai.model.data
for dl in [0,1]:
    new_x, new_y = [],[]
    dl_len = len(bonsai.model.data[dl].dataset)
    for i,(x,y) in enumerate(bonsai.model.data[dl].dataset):
        if i%100==0:
            print("\r{:>6,}/{:<6,}".format(i,dl_len),end="")
        rot = np.random.choice([0,1,2,3],1)[0]
        rot_x = torch.rot90(x,rot,[1,2])
        new_x.append(rot_x)
        new_y.append(rot)
    new_x = torch.stack(new_x)
    new_y = torch.tensor(new_y)
    data = list(zip(new_x,new_y))
    bonsai.model.data[dl] = torch.utils.data.DataLoader(data,
                                                        batch_size=hypers['batch_size'],
                                                        shuffle=True)
    print()

49,900/50,000
 9,900/10,000


In [11]:
print(bonsai.model)
bonsai.model.prune

=========================== NETWORK ===========================
================ Scottsdale Galilee Charles_r3 ================
                     :     Dim      :    Params    :   Comp   
Initializer          :              :     180      :          
Cell 0  (Normal)     :   36 x 32    :    60,338    :   38.5%  
Cell 1  (Normal)     :   36 x 32    :    51,811    :   38.5%  
 ↳ Aux Tower         :              :     148      :          
Cell 2  (Reduction)  :   72 x 16    :   208,483    :   38.2%  
Cell 3  (Normal)     :   72 x 16    :   217,556    :   38.5%  
Cell 4  (Normal)     :   72 x 16    :   192,361    :   38.5%  
 ↳ Aux Tower         :              :     292      :          
Cell 5  (Reduction)  :  144 x 8     :   782,542    :   38.5%  
 ↳ Aux Tower         :              :     580      :          
Cell 6  (Normal)     :  144 x 8     :   764,829    :   38.5%  
Cell 7  (Normal)     :  144 x 8     :   915,016    :   38.5%  
 ↳ Classifier        :              :     580      : 

{'edge': False, 'input': False}

In [12]:
bonsai.train()

=========================== NETWORK ===========================
================ Scottsdale Galilee Charles_r3 ================
                     :     Dim      :    Params    :   Comp   
Initializer          :              :     180      :          
Cell 0  (Normal)     :   36 x 32    :    60,338    :   38.5%  
Cell 1  (Normal)     :   36 x 32    :    51,811    :   38.5%  
 ↳ Aux Tower         :              :     148      :          
Cell 2  (Reduction)  :   72 x 16    :   208,483    :   38.2%  
Cell 3  (Normal)     :   72 x 16    :   217,556    :   38.5%  
Cell 4  (Normal)     :   72 x 16    :   192,361    :   38.5%  
 ↳ Aux Tower         :              :     292      :          
Cell 5  (Reduction)  :  144 x 8     :   782,542    :   38.5%  
 ↳ Aux Tower         :              :     580      :          
Cell 6  (Normal)     :  144 x 8     :   764,829    :   38.5%  
Cell 7  (Normal)     :  144 x 8     :   915,016    :   38.5%  
 ↳ Classifier        :              :     580      : 

Last Tower Test  Corrects: Top-1: 72.51%, 16.96s

Adjusting lrs to [0.00996057350657239, 0.00996057350657239]
06/21/2020 03:54 AM
Train Epoch: 24  [12512 /50000  (100%)]	Per Epoch: 4m,26s , Alloc: 6.97GiB, Data T: 0.461 , Op T: 395.001
Train Corrects: Top-1: 67.65%, 6m,35s
Last Tower Test  Corrects: Top-1: 72.21%, 16.96s

Adjusting lrs to [0.009957224306869053, 0.009957224306869053]
06/21/2020 04:01 AM
Train Epoch: 25  [12512 /50000  (100%)]	Per Epoch: 4m,19s , Alloc: 6.97GiB, Data T: 0.469 , Op T: 395.179
Train Corrects: Top-1: 68.25%, 6m,35s
Last Tower Test  Corrects: Top-1: 72.82%, 16.96s

Adjusting lrs to [0.009953739202357217, 0.009953739202357217]
06/21/2020 04:08 AM
Train Epoch: 26  [12512 /50000  (100%)]	Per Epoch: 4m,19s , Alloc: 6.97GiB, Data T: 0.456 , Op T: 396.263
Train Corrects: Top-1: 68.42%, 6m,37s
Last Tower Test  Corrects: Top-1: 73.72%, 16.96s

Adjusting lrs to [0.009950118288582789, 0.009950118288582789]
06/21/2020 04:15 AM
Train Epoch: 27  [12512 /50000  (100%)]	Pe

Train Epoch: 52  [12512 /50000  (100%)]	Per Epoch: 4m,17s , Alloc: 6.97GiB, Data T: 0.466 , Op T: 393.014
Train Corrects: Top-1: 77.33%, 6m,33s
Last Tower Test  Corrects: Top-1: 80.37%, 16.96s

Adjusting lrs to [0.009808706547746057, 0.009808706547746057]
06/21/2020 07:13 AM
Train Epoch: 53  [12512 /50000  (100%)]	Per Epoch: 3m,55s , Alloc: 6.97GiB, Data T: 0.456 , Op T: 390.831
Train Corrects: Top-1: 77.50%, 6m,31s
Last Tower Test  Corrects: Top-1: 80.43%, 16.95s

Adjusting lrs to [0.009801468428384716, 0.009801468428384716]
06/21/2020 07:20 AM
Train Epoch: 54  [12512 /50000  (100%)]	Per Epoch: 3m,56s , Alloc: 6.97GiB, Data T: 0.449 , Op T: 389.423
Train Corrects: Top-1: 77.77%, 6m,30s
Last Tower Test  Corrects: Top-1: 80.30%, 16.95s

Adjusting lrs to [0.009794098674340966, 0.009794098674340966]
06/21/2020 07:27 AM
Train Epoch: 55  [12512 /50000  (100%)]	Per Epoch: 4m,26s , Alloc: 6.97GiB, Data T: 0.463 , Op T: 390.370
Train Corrects: Top-1: 78.24%, 6m,31s
Last Tower Test  Corrects: T

Train Epoch: 81  [12512 /50000  (100%)]	Per Epoch: 4m,24s , Alloc: 6.97GiB, Data T: 0.459 , Op T: 393.905
Train Corrects: Top-1: 82.51%, 6m,34s
Last Tower Test  Corrects: Top-1: 82.14%, 16.95s

Adjusting lrs to [0.009546180545235343, 0.009546180545235343]
06/21/2020 10:32 AM
Train Epoch: 82  [12512 /50000  (100%)]	Per Epoch: 4m,14s , Alloc: 6.97GiB, Data T: 0.466 , Op T: 393.844
Train Corrects: Top-1: 82.78%, 6m,34s
Last Tower Test  Corrects: Top-1: 82.81%, 16.95s

Adjusting lrs to [0.009535220071457325, 0.009535220071457325]
06/21/2020 10:38 AM
Train Epoch: 83  [12512 /50000  (100%)]	Per Epoch: 4m,17s , Alloc: 6.97GiB, Data T: 0.452 , Op T: 391.381
Train Corrects: Top-1: 83.00%, 6m,32s
Last Tower Test  Corrects: Top-1: 82.98%, 17.04s

Adjusting lrs to [0.009524135262330098, 0.009524135262330098]
06/21/2020 10:45 AM
Train Epoch: 84  [12512 /50000  (100%)]	Per Epoch: 4m,15s , Alloc: 6.97GiB, Data T: 0.459 , Op T: 391.542
Train Corrects: Top-1: 82.93%, 6m,32s
Last Tower Test  Corrects: T

Train Epoch: 110 [12512 /50000  (100%)]	Per Epoch: 4m,19s , Alloc: 6.97GiB, Data T: 0.461 , Op T: 394.266
Train Corrects: Top-1: 85.89%, 6m,35s
Last Tower Test  Corrects: Top-1: 81.95%, 16.96s

Adjusting lrs to [0.009179036806841352, 0.009179036806841352]
06/21/2020 01:50 PM
Train Epoch: 111 [12512 /50000  (100%)]	Per Epoch: 4m,23s , Alloc: 6.97GiB, Data T: 0.455 , Op T: 394.613
Train Corrects: Top-1: 86.09%, 6m,35s
Last Tower Test  Corrects: Top-1: 83.03%, 16.96s

Adjusting lrs to [0.009164606203550498, 0.009164606203550498]
06/21/2020 01:57 PM
Train Epoch: 112 [12512 /50000  (100%)]	Per Epoch: 4m,22s , Alloc: 6.97GiB, Data T: 0.470 , Op T: 395.974
Train Corrects: Top-1: 86.01%, 6m,36s
Last Tower Test  Corrects: Top-1: 83.81%, 16.96s

Adjusting lrs to [0.009150061425476837, 0.009150061425476837]
06/21/2020 02:04 PM
Train Epoch: 113 [12512 /50000  (100%)]	Per Epoch: 3m,56s , Alloc: 6.97GiB, Data T: 0.453 , Op T: 389.569
Train Corrects: Top-1: 85.95%, 6m,30s
Last Tower Test  Corrects: T

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/campus.ncl.ac.uk/b6070424/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3291, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-12-16464926ee4b>", line 1, in <module>
    bonsai.train()
  File "/home/campus.ncl.ac.uk/b6070424/Documents/Programming/Python/bonsai_net/bonsai/nas.py", line 158, in train
    nas_schedule=self.hypers['nas_schedule'])
  File "/home/campus.ncl.ac.uk/b6070424/Documents/Programming/Python/bonsai_net/bonsai/trainers.py", line 301, in full_train
    train(model, device, criterion=criterion, optimizer=optimizer, epoch=epoch, comp_lambda=comp_lambda, **kwargs)
  File "/home/campus.ncl.ac.uk/b6070424/Documents/Programming/Python/bonsai_net/bonsai/trainers.py", line 152, in train
    loss.backward()
  File "/home/campus.ncl.ac.uk/b6070424/.local/lib/python3.6/site-packages/torch/tensor.py", line 198, in backward
    torch.autograd.backward(self, gradien

KeyboardInterrupt: 

In [ ]:
bonsai.model.rot_towers = bonsai.model.towers

In [38]:
bonsai.model.data = load_data(hypers['batch_size'],hypers['dataset']['name'])[0]

In [48]:
for parameter in bonsai.model.parameters():
    parameter.requires_grad=False

bonsai.model.pattern_params = {}
bonsai.model.towers = nn.ModuleDict({'Classifier':Classifier(7,True,[None,144],10)})
bonsai.model.pattern_params['init'] = bonsai.model.towers.parameters()
bonsai.model.lr_schedulers['init'] = LRScheduler(16,.1)

In [49]:
print(bonsai.model)

=========================== NETWORK ===========================
================ Scottsdale Galilee Charles_r3 ================
                     :     Dim      :    Params    :   Comp   
Initializer          :              :      0       :          
Cell 0  (Normal)     :   36 x 32    :      0       :   38.5%  
Cell 1  (Normal)     :   36 x 32    :      0       :   38.5%  
Cell 2  (Reduction)  :   72 x 16    :      0       :   38.2%  
Cell 3  (Normal)     :   72 x 16    :      0       :   38.5%  
Cell 4  (Normal)     :   72 x 16    :      0       :   38.5%  
Cell 5  (Reduction)  :  144 x 8     :      0       :   38.5%  
Cell 6  (Normal)     :  144 x 8     :      0       :   38.5%  
Cell 7  (Normal)     :  144 x 8     :      0       :   38.5%  
 ↳ Classifier        :              :    1,450     :          
Total                :              :    1,450     :   38.5%  



In [54]:
bonsai.train()

INFO:training_logger:=== NEW FULL TRAIN ===
INFO:training_logger:Starting at 2020-06-21 15:17:20.835615
INFO:jn_out:Starting at 2020-06-21 15:17:20.835615
INFO:training_logger:ID: 'Scottsdale Galilee Charles_r3', Dim: torch.Size([64, 3, 32, 32]), Classes: 4, Scale: 36, N: 8, Patterns: <generator object looping_generator at 0x7f67d3ee3fc0>, Nodes: 4, Pruners: {'edge': False, 'input': False}
INFO:training_logger:=== EPOCH 0 ===
INFO:jn_out:06/21/2020 03:17 PM
INFO:jn_out:Init:    6.97GiB
INFO:jn_out:0:    6.97GiB
INFO:jn_out:1:    6.97GiB
INFO:jn_out:2:    6.97GiB
INFO:jn_out:3:    6.97GiB
INFO:jn_out:4:    6.97GiB
INFO:jn_out:5:    6.97GiB
INFO:jn_out:6:    6.97GiB
INFO:jn_out:7:    6.97GiB


=========================== NETWORK ===========================
================ Scottsdale Galilee Charles_r3 ================
                     :     Dim      :    Params    :   Comp   
Initializer          :              :      0       :          
Cell 0  (Normal)     :   36 x 32    :      0       :   38.5%  
Cell 1  (Normal)     :   36 x 32    :      0       :   38.5%  
Cell 2  (Reduction)  :   72 x 16    :      0       :   38.2%  
Cell 3  (Normal)     :   72 x 16    :      0       :   38.5%  
Cell 4  (Normal)     :   72 x 16    :      0       :   38.5%  
Cell 5  (Reduction)  :  144 x 8     :      0       :   38.5%  
Cell 6  (Normal)     :  144 x 8     :      0       :   38.5%  
Cell 7  (Normal)     :  144 x 8     :      0       :   38.5%  
 ↳ Classifier        :              :    1,450     :          
Total                :              :    1,450     :   38.5%  

=== Training Scottsdale Galilee Charles_r3 ===
Starting at 2020-06-21 15:17:20.835615
06/21/2020 03:17 PM
Init:    

INFO:jn_out:Classifier:    6.97GiB


tensor([6, 6, 6, 7, 0, 6, 5, 9, 6, 4, 9, 9, 5, 4, 9, 1, 7, 1, 6, 0, 6, 6, 5, 6,
        1, 9, 7, 6, 7, 1, 1, 6, 1, 4, 0, 4, 6, 6, 0, 7, 0, 1, 9, 1, 6, 6, 6, 1,
        7, 9, 4, 6, 6, 7, 6, 4, 6, 8, 9, 6, 6, 1, 6, 1], device='cuda:0',
       grad_fn=<NotImplemented>)


INFO:jn_out:Train Epoch: 0   [64    /50000  (0%)]	Per Epoch: 1m,48s , Alloc: 6.97GiB, Data T: 0.000 , Op T: 0.000 |carr_ret|


...

INFO:jn_out:Train Epoch: 0   [4544  /50000  (9%)]	Per Epoch: 1m,42s , Alloc: 6.97GiB, Data T: 0.886 , Op T: 9.160 |carr_ret|


tensor([7, 6, 6, 6, 6, 7, 6, 6, 1, 4, 6, 4, 9, 2, 9, 1, 4, 1, 9, 5, 7, 1, 6, 6,
        6, 4, 4, 1, 1, 0, 1, 0, 4, 9, 6, 6, 1, 4, 9, 9, 7, 6, 6, 5, 1, 6, 9, 0,
        9, 5, 6, 5, 1, 6, 9, 6, 6, 4, 7, 6, 6, 0, 6, 3], device='cuda:0',
       grad_fn=<NotImplemented>)
tensor([3, 8, 9, 0, 7, 6, 9, 6, 6, 9, 7, 0, 6, 5, 6, 1, 6, 4, 6, 6, 6, 1, 4, 7,
        7, 6, 3, 7, 4, 6, 6, 7, 6, 6, 7, 6, 9, 6, 7, 6, 1, 1, 7, 1, 6, 9, 4, 9,
        6, 6, 6, 9, 9, 6, 7, 9, 2, 7, 6, 6, 6, 9, 7, 6], device='cuda:0',
       grad_fn=<NotImplemented>)
tensor([4, 7, 0, 4, 0, 9, 9, 6, 1, 7, 6, 1, 6, 9, 9, 1, 9, 6, 6, 6, 6, 6, 6, 4,
        6, 5, 6, 6, 6, 6, 4, 9, 9, 0, 1, 1, 9, 9, 4, 4, 6, 7, 1, 7, 7, 9, 7, 4,
        7, 6, 6, 9, 6, 6, 6, 3, 6, 4, 6, 0, 5, 7, 7, 6], device='cuda:0',
       grad_fn=<NotImplemented>)
tensor([8, 5, 7, 6, 6, 7, 9, 9, 1, 1, 6, 6, 5, 5, 6, 5, 5, 1, 6, 6, 6, 7, 6, 6,
        6, 9, 6, 7, 9, 3, 9, 1, 4, 7, 6, 9, 1, 4, 1, 6, 9, 9, 1, 9, 5, 7, 7, 7,
        1, 6, 7, 1, 6, 9, 9, 9, 6, 6, 6

INFO:jn_out:Train Epoch: 0   [5184  /50000  (10%)]	Per Epoch: 1m,42s , Alloc: 6.97GiB, Data T: 1.010 , Op T: 10.468|carr_ret|


tensor([6, 6, 1, 1, 5, 4, 6, 6, 5, 7, 9, 9, 9, 7, 6, 6, 4, 6, 6, 6, 6, 6, 6, 6,
        6, 7, 0, 1, 6, 9, 6, 0, 0, 5, 4, 1, 6, 7, 9, 6, 5, 5, 1, 6, 1, 6, 1, 7,
        7, 9, 5, 0, 9, 2, 1, 6, 5, 6, 4, 1, 6, 1, 7, 1], device='cuda:0',
       grad_fn=<NotImplemented>)
tensor([6, 7, 6, 1, 5, 6, 7, 6, 1, 1, 8, 6, 9, 9, 5, 6, 7, 3, 5, 9, 7, 1, 6, 6,
        6, 1, 9, 7, 7, 1, 1, 6, 7, 6, 6, 6, 0, 3, 4, 1, 6, 7, 9, 9, 0, 1, 6, 0,
        1, 3, 6, 4, 6, 6, 6, 0, 1, 6, 1, 4, 3, 1, 1, 6], device='cuda:0',
       grad_fn=<NotImplemented>)
tensor([7, 6, 5, 7, 9, 1, 9, 6, 3, 6, 6, 4, 1, 6, 6, 6, 0, 6, 1, 1, 9, 6, 5, 6,
        9, 7, 1, 6, 0, 1, 6, 7, 8, 6, 4, 9, 1, 5, 6, 4, 1, 9, 1, 7, 6, 4, 0, 6,
        1, 7, 6, 6, 6, 6, 7, 9, 7, 6, 4, 1, 8, 7, 7, 7], device='cuda:0',
       grad_fn=<NotImplemented>)
tensor([5, 9, 6, 0, 6, 1, 9, 6, 7, 6, 1, 6, 6, 3, 4, 7, 1, 6, 6, 4, 1, 3, 9, 0,
        7, 9, 9, 9, 6, 7, 8, 4, 4, 0, 0, 6, 6, 6, 2, 0, 1, 1, 3, 9, 6, 6, 7, 7,
        1, 6, 4, 1, 1, 1, 6, 6, 7, 6, 5

KeyboardInterrupt: 

In [36]:
for x,y in bonsai.model.data[1]:
    print(y)

tensor([2, 2, 1, 2, 0, 3, 0, 2, 0, 0, 3, 1, 2, 3, 0, 1, 2, 3, 1, 0, 3, 3, 2, 3,
        3, 3, 3, 0, 1, 3, 2, 0, 1, 1, 3, 2, 2, 3, 2, 2, 2, 1, 1, 0, 0, 3, 1, 0,
        2, 1, 0, 3, 1, 1, 2, 1, 2, 2, 2, 0, 2, 1, 3, 2])
tensor([1, 1, 2, 2, 3, 0, 0, 1, 1, 2, 0, 1, 1, 1, 3, 1, 2, 1, 0, 0, 3, 1, 0, 1,
        3, 2, 2, 2, 2, 1, 1, 3, 0, 1, 1, 0, 1, 3, 3, 0, 0, 2, 3, 1, 3, 1, 2, 3,
        1, 3, 1, 2, 1, 2, 0, 2, 2, 1, 1, 3, 2, 0, 2, 2])
tensor([1, 1, 2, 2, 2, 0, 1, 3, 2, 0, 3, 2, 0, 2, 0, 1, 3, 1, 2, 3, 3, 0, 3, 2,
        0, 1, 2, 0, 1, 2, 2, 3, 2, 3, 3, 2, 1, 1, 3, 1, 3, 0, 1, 3, 0, 3, 0, 2,
        0, 2, 1, 0, 3, 2, 3, 3, 1, 2, 2, 0, 2, 1, 3, 1])
tensor([0, 3, 0, 1, 2, 0, 2, 3, 1, 0, 2, 1, 2, 0, 3, 1, 3, 3, 2, 1, 0, 2, 0, 3,
        3, 0, 0, 3, 3, 0, 3, 1, 1, 0, 0, 1, 3, 2, 0, 0, 0, 3, 2, 2, 3, 0, 2, 1,
        1, 2, 0, 0, 1, 3, 0, 3, 2, 3, 2, 2, 1, 3, 0, 3])
tensor([1, 0, 1, 1, 3, 0, 0, 0, 3, 2, 2, 3, 0, 3, 0, 1, 2, 2, 2, 0, 0, 2, 1, 1,
        2, 0, 3, 0, 1, 0, 0, 3, 2, 0, 2, 1, 3, 3, 2,

# Cool Graphs

In [43]:
import collections
import matplotlib.pyplot as plt
import numpy as np
from bonsai.chroma import color_create
plt.style.use('material')

colors = color_create()
for i,cell in enumerate(bonsai.model.cells):
    
    plt.figure(figsize=(16,9), dpi=100)
    for k,edge in cell.edges.items():
        for op in edge.ops:
            on = op.name
            op_hist = []
            for hist in op.pruner.weight_history_history:
                for val in hist:
                    if type(val) is float:
                       op_hist.append(int(val))
                    else:
                        op_hist += [int(x) for x in val]
                    
                print([x if type(x) is float else x for x in hist])
                op_hist += [int(x) for x in hist]
            print(op_hist)
        #plt.plot(np.cumsum(hist),c=colors[on]['hex'],label=on)
        #plt.scatter(len(hist), np.cumsum(hist)[-1], marker='x', c=colors[on]['hex'])
    #plt.show()
    break

1.0
1.0
1.0
1.0
[1.0, 1.0, 1.0, 1.0, [...]]


TypeError: int() argument must be a string, a bytes-like object or a number, not 'list'

<Figure size 1600x900 with 0 Axes>

In [44]:
for i, cell in enumerate(bonsai.model.cells):
    print("===== Cell {} =====".format(i))
    for k,edge in cell.edges.items():
        print("=== Edge {}".format(k))
        for op in edge.ops:
            print(op.name)
            print([x for x in op.pruner.weight_history_history])

===== Cell 0 =====
=== Edge x->0
Identity
[[1.0, 1.0, 1.0, 1.0, [...]], [1.0, 1.0, 1.0, 1.0, [...]], [1.0, 1.0, 1.0, 1.0, [...]], [1.0, 1.0, 1.0, 1.0, [...]], [1.0, 1.0, 1.0, 1.0, [...]], [1.0, 1.0, 1.0, 1.0, [...]], [1.0, 1.0, 1.0, 1.0, [...]], [1.0, 1.0, 1.0, 1.0, [...]], [1.0, 1.0, 1.0, 1.0, [...]], [1.0, 1.0, 1.0, 1.0, [...]], [1.0, 1.0, 1.0, 1.0, [...]], [1.0, 1.0, 1.0, 1.0, [...]], [1.0, 1.0, 1.0, 1.0, [...]], [1.0, 1.0, 1.0, 1.0, [...]], [1.0, 1.0, 1.0, 1.0, [...]], [1.0, 1.0, 1.0, 1.0, [...]], [1.0, 1.0, 1.0, 1.0, [...]], [1.0, 1.0, 1.0, 1.0, [...]], [1.0, 1.0, 1.0, 1.0, [...]], [1.0, 1.0, 1.0, 1.0, [...]], [1.0, 1.0, 1.0, 1.0, [...]], [1.0, 1.0, 1.0, 1.0, [...]], [1.0, 1.0, 1.0, 1.0, [...]], [1.0, 1.0, 1.0, 1.0, [...]], [1.0, 1.0, 1.0, 1.0, [...]], [1.0, 1.0, 1.0, 1.0, [...]], [1.0, 1.0, 1.0, 1.0, [...]], [1.0, 1.0, 1.0, 1.0, [...]], [1.0, 1.0, 1.0, 1.0, [...]], [1.0, 1.0, 1.0, 1.0, [...]], [1.0, 1.0, 1.0, 1.0, [...]], [1.0, 1.0, 1.0, 1.0, [...]], [1.0, 1.0, 1.0, 1.0, [...]], 

Sep_Conv_3x3
[[1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0], [1.0, 0.0, 1.0, 1.0], [1.0, 0.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0], [0.0, 0.0, 1.0, 1.0], [1.0, 1.0, 1.0, 0.0], [1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0], [0.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0], [1.0, 0.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.

In [ ]:
#bonsai.random_search(1)

In [ ]:
#bonsai.random_search(3, e_c=.575,i_c=1.5)